In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import hyperspy.api as hs

In [3]:

data = hs.load("Datapath/lowloss_0p1eV_001.dm4")

In [5]:
eels = data[-1]

In [6]:
eels.plot()

In [17]:
eels.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       ├── aperture_size = 5.0
│       │       ├── collection_angle = 66.0
│       │       ├── dwell_time = 0.00019
│       │       ├── frame_number = 1
│       │       └── spectrometer = GIF Quantum ER
│       ├── Stage
│       │   ├── tilt_alpha = 2.4700000286102295
│       │   ├── tilt_beta = 0.0
│       │   ├── x = -0.4903726875
│       │   ├── y = 0.10138610156250001
│       │   └── z = -0.07525260156250001
│       ├── acquisition_mode = STEM
│       ├── beam_current = 0.0
│       ├── beam_energy = 200.0
│       ├── camera_length = 15.0
│       ├── convergence_angle = 27.42
│       ├── magnification = 25000.0
│       └── microscope = JEM-ARM200F
├── General
│   ├── FileIO
│   │   └── 0
│   │       ├── hyperspy_version = 1.7.1
│   │       ├── io_plugin = hyperspy.io_plugins.digital_micrograph
│   │       ├── operation = load
│   │       └── timestamp = 2023-06-10T15:43:26.555354+02:00
│   ├── date = 2023-11-01
│   ├── original_filename = lowloss_0p1eV_001.dm4
│   ├── time = 16:57:35
│   └── title = EELS Spectrum Image
├── Sample
│   └── description = Ag_YVC016_FIB2
└── Signal
    ├── Noise_properties
    │   └── Variance_linear_model
    │       ├── gain_factor = 1.0
    │       └── gain_offset = 0.0
    ├── quantity = Intensity (Counts)
    └── signal_type = EELS

In [13]:

E0  =200
F = (1 + E0/1022)/(1+E0/511)**2

rho = 7.29

alpha = 27.42 #convergence angle
beta = 66 # collection angle

delta =  np.abs(alpha**2 - beta**2)

theta_e = 5.5*rho**(0.3)/(F*E0)
theta_c = 20

upper = alpha**2 + beta**2 + 2*theta_e**2 + delta**2
lower = alpha**2 + beta**2 + 2*theta_c**2 + delta**2

lam = (200*F*E0)/(11*rho**(0.3)) /(np.log((upper/lower * theta_c**2/theta_e**2)))
lam

112.27337222021647

In [8]:
eels.data = eels.data[::-1,::-1,:]

In [50]:
eels.plot()

In [10]:
zlp = eels.estimate_zero_loss_peak_centre()

In [11]:
thickness_map = eels.estimate_thickness(threshold=zlp + 3, density=7.39)

[########################################] | 100% Completed |  1.0s


In [12]:
thickness_sig = hs.signals.Signal2D(thickness_map.data)

thickness_sig.axes_manager[0].scale = thickness_map.axes_manager[0].scale
thickness_sig.axes_manager[1].scale = thickness_map.axes_manager[1].scale
thickness_sig.axes_manager[0].units = thickness_map.axes_manager[0].units
thickness_sig.axes_manager[1].units = thickness_map.axes_manager[1].units

In [19]:
thickness_sig.plot(title="", vmin = 0,vmax=100)
plt.xlabel("")
plt.ylabel("")
plt.tight_layout()


In [80]:
thickness_rectangle = hs.roi.RectangularROI(left=2.01424, top=0.533182, right=2.51781, bottom=0.94788)
thickness_sig.plot()

thickness_rectangle.add_widget(thickness_sig)

In [83]:
print(thickness_rectangle(thickness_map).data.mean())
print(thickness_rectangle(thickness_map).data.std())

66.51841
4.8045616
